In [1]:
import csv, re, os

def generate_insert_statements(
    csv_file, 
    create_stream_statement, 
    output_file, 
    table_name):
    # Extract content within parentheses
    stream_content = re.search(r'\((.*?)\)', create_stream_statement, re.DOTALL).group(1)
    
    # Split content into individual column definitions
    columns = re.findall(r'([a-zA-Z_]+)\s+([a-zA-Z_]+)', stream_content)

    # Extract column names and data types, stripping BOM if present
    column_names_create_stream = [col[0].strip() for col in columns]
    data_types_create_stream = {col[0].strip(): col[1] for col in columns}

    print(column_names_create_stream)
    print(data_types_create_stream)
    
    # Open the CSV file
    with open(csv_file, newline='', encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        insert_statements = []

        # Iterate over each row in the CSV
        for row in reader:
            columns = []
            values = []

            # Iterate over each column in the row
            for field, value in row.items():
                if field in column_names_create_stream:
                    # Column name exists in create_stream_statement
                    data_type = data_types_create_stream[field]
                    if value is None:
                        columns.append(field)
                        values.append("")
                    else:
                        if data_type == "DOUBLE" or data_type == "INTEGER" or data_type == "BIGINT":
                            value_str = str(value)
                            if value_str.strip() == "":
                                columns.append(field)
                                values.append("")
                            else:
                                columns.append(field)
                                values.append(value_str)
                        elif data_type == "TIMESTAMP" or data_type == "DATE":
                            columns.append(field)
                            values.append("'" + str(value) + "'")
                        elif data_type == "VARCHAR":
                            columns.append(field)
                            values.append("'" + value.replace("'", "''") + "'")

            # Construct the INSERT INTO statement with the specified table name
            insert_statement = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(values)});"
            insert_statements.append(insert_statement)

    # Write insert statements to output file
    with open(output_file, 'w') as f:
        for statement in insert_statements:
            f.write(statement + '\n')

In [2]:
create_statements = [
    """CREATE STREAM daily_activity (
        id BIGINT KEY, 
        activity_date VARCHAR, 
        total_steps DOUBLE, 
        total_distance DOUBLE, 
        tracker_distance DOUBLE, 
        logged_activities_distance INTEGER, 
        very_active_distance DOUBLE, 
        moderately_active_distance DOUBLE, 
        light_active_distance DOUBLE, 
        sedentary_active_distance DOUBLE,
        very_active_minutes INTEGER, 
        fairly_active_minutes INTEGER, 
        lightly_active_minutes INTEGER, 
        sedentary_minutes INTEGER, 
        calories INTEGER
    )""", 
    """CREATE STREAM daily_calories (id BIGINT KEY, activity_day VARCHAR, calories INTEGER)""", 
    """CREATE STREAM daily_intensities (id BIGINT KEY, activity_day VARCHAR, sedentary_minutes INTEGER, lightly_active_minutes INTEGER, fairly_active_minutes INTEGER, very_active_minutes INTEGER, sedentary_active_distance DOUBLE, light_active_distance DOUBLE, moderately_active_distance DOUBLE, very_active_distance DOUBLE)""", 
    """CREATE STREAM daily_steps (id BIGINT KEY, activity_day VARCHAR, step_total INTEGER)""",
    """CREATE STREAM hourly_calories (id BIGINT KEY, activity_hour VARCHAR, calories INTEGER)""",
    """CREATE STREAM hourly_intensities (id BIGINT KEY, activity_hour VARCHAR, total_intensity INTEGER, average_intensity DOUBLE)""",
    """CREATE STREAM hourly_steps (id BIGINT KEY, activity_hour VARCHAR, step_total INTEGER)""",
    """CREATE STREAM minute_calories_narrow (id BIGINT KEY, activity_minute VARCHAR, calories DOUBLE)""",
    """CREATE STREAM minute_intensities_narrow (id BIGINT KEY, activity_minute INTEGER, intensity INTEGER)""",
    """CREATE STREAM minute_METs_narrow (id BIGINT KEY, activity_minute VARCHAR, mets INTEGER)""",
    """CREATE STREAM minute_sleep (id BIGINT KEY, date_and_time VARCHAR, sleep_minutes INTEGER, log_id BIGINT)""",
    """CREATE STREAM minute_steps_narrow (id BIGINT KEY, activity_minute VARCHAR, num_of_steps INTEGER)""",
    """CREATE STREAM sleep_day (id BIGINT KEY, sleep_day VARCHAR, total_sleep_records INTEGER, total_minutes_asleep INTEGER, total_time_in_bed INTEGER)"""
]

# remove all instances of ' KEY'
create_statements = [item.replace(' KEY', '') for item in create_statements]

In [3]:
folder_name = '/Users/briandunn/Desktop/Kafka Projects/Fitbit Fitness Data/data'

"""
Remove these:
    - minute_calories_wide
    - heartrate_seconds
    - minute_steps_wide
    - minute_intensities_wide
    - weight_log_info
"""

csv_files = [
    'daily_activity',
    'daily_calories',
    'daily_intensities',
    'daily_steps',
    'hourly_calories',
    'hourly_intensities',
    'hourly_steps',
    'minute_calories_narrow',
    'minute_intensities_narrow',
    'minute_METs_narrow',
    'minute_sleep',
    'minute_steps_narrow',
    'sleep_day'
]

csv_files = [item.strip() for item in csv_files]

input_data_file_prefix = '/Users/briandunn/Desktop/Kafka/ksql/Fitbit Fitness Data/data'

output_file_prefix = '/Users/briandunn/Desktop/Kafka/ksql/Fitbit Fitness Data/statements/producers'

for i in range(len(csv_files)):
    input_data_file = os.path.join(input_data_file_prefix, csv_files[i] + '.csv')
    output_file_location = os.path.join(output_file_prefix, csv_files[i] + '_inserts.sql')
    
    generate_insert_statements(
        input_data_file, 
        create_statements[i], 
        output_file_location,
        csv_files[i]
        )

/Users/briandunn/Desktop/Kafka/ksql/Fitbit Fitness Data/data/daily_activity.csv
CREATE STREAM daily_activity (
        id BIGINT, 
        activity_date VARCHAR, 
        total_steps DOUBLE, 
        total_distance DOUBLE, 
        tracker_distance DOUBLE, 
        logged_activities_distance INTEGER, 
        very_active_distance DOUBLE, 
        moderately_active_distance DOUBLE, 
        light_active_distance DOUBLE, 
        sedentary_active_distance INTEGER, 
        very_active_minutes INTEGER, 
        fairly_active_minutes INTEGER, 
        lightly_active_minutes INTEGER, 
        sedentary_minutes INTEGER, 
        calories INTEGER
    )
/Users/briandunn/Desktop/Kafka/ksql/Fitbit Fitness Data/statements/producers/daily_activity_inserts.sql
daily_activity

['id', 'activity_date', 'total_steps', 'total_distance', 'tracker_distance', 'logged_activities_distance', 'very_active_distance', 'moderately_active_distance', 'light_active_distance', 'sedentary_active_distance', 'very_activ